<a href="https://colab.research.google.com/github/zsenof/Systeme_recommandation_films_creuse/blob/main/COLAB__jointure_IMDB_TMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Sprint 2 : APPROPRIATION, EXPLORATION DES DONNEES ET NETTOYAGE**

***Outils : SQL, Pandas, Matplotlib, Seaborn***

**1. Récupération des fichiers de données IMDB [datasets IMDb](https://datasets.imdbws.com/)**

In [ ]:
import pandas as pd
import duckdb
link1= "https://datasets.imdbws.com/name.basics.tsv.gz"
link2= "https://datasets.imdbws.com/title.akas.tsv.gz"
link3= "https://datasets.imdbws.com/title.basics.tsv.gz"
link4= "https://datasets.imdbws.com/title.crew.tsv.gz"
link5= "https://datasets.imdbws.com/title.episode.tsv.gz"
link6= "https://datasets.imdbws.com/title.principals.tsv.gz"
link7= "https://datasets.imdbws.com/title.ratings.tsv.gz"



---



**2. Choix de la dataset [title.basics](https://datasets.imdbws.com/title.basics.tsv.gz) pour réaliser les filtres suivants définis dans les KPI :**


*  titleType = movie
*  genres = drama et comedy,Drama
*  startYear
*  isAdult

**Jointure de cette table avec la dataset [title.ratings](https://datasets.imdbws.com/title.ratings.tsv.gz) pour obtenir la dataframe comportant les votes par film.**


In [ ]:
# Chargement en Duck DB de name.basics -  link3= "https://datasets.imdbws.com/title.basics.tsv.gz"

con = duckdb.connect()

data_initial = con.execute(f"""

    WITH result_title_basics AS (
      SELECT *
      FROM read_csv_auto('https://datasets.imdbws.com/title.basics.tsv.gz', delim='\t', quote='', ignore_errors=true)
      WHERE titleType = 'movie'
      AND ((genres LIKE 'Drama')OR (genres LIKE 'Comedy,Drama'))
      AND ((startYear BETWEEN '1960' AND '1979') OR (startYear BETWEEN '2022' AND '2024'))
      AND isAdult = 0
      )
    SELECT *
    FROM read_csv_auto('https://datasets.imdbws.com/title.ratings.tsv.gz', delim='\t', quote='', ignore_errors=true) as title_rating
    RIGHT JOIN result_title_basics
    ON result_title_basics.tconst= title_RATING.tconst



""").fetchdf()

data_initial

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,tconst,averageRating,numVotes,tconst_1,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0052614,7.0,78.0,tt0052614,movie,Le bel âge,Le bel âge,0,1960,\N,100,"Comedy,Drama"
1,tt0052685,5.2,12.0,tt0052685,movie,El cerro de los locos,El cerro de los locos,0,1960,\N,88,"Comedy,Drama"
2,tt0052788,6.6,20.0,tt0052788,movie,Una estrella y dos estrellados,Una estrella y dos estrellados,0,1960,\N,80,"Comedy,Drama"
3,tt0053570,7.6,1449.0,tt0053570,movie,Adua e le compagne,Adua e le compagne,0,1960,\N,106,"Comedy,Drama"
4,tt0053579,7.9,6348.0,tt0053579,movie,Late Autumn,Akibiyori,0,1960,\N,128,"Comedy,Drama"
...,...,...,...,...,...,...,...,...,...,...,...,...
26354,None,NaN,NaN,tt0312839,movie,Ji-mi Is Not Sad,Ji-mineun seulpeuji anhda,0,1963,\N,\N,Drama
26355,None,NaN,NaN,tt0312840,movie,Homeless Angel,Jibeobtneun cheonsa,0,1966,\N,\N,Drama
26356,None,NaN,NaN,tt0312850,movie,Pebbles,Joyakdol,0,1975,\N,90,Drama
26357,None,NaN,NaN,tt0312856,movie,Juventud desnuda,Juventud desnuda,0,1971,\N,85,Drama


**3. Exploration de la dataframe initial créée**

In [ ]:
data_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26359 entries, 0 to 26358
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          14407 non-null  object 
 1   averageRating   14407 non-null  float64
 2   numVotes        14407 non-null  float64
 3   tconst_1        26359 non-null  object 
 4   titleType       26359 non-null  object 
 5   primaryTitle    26359 non-null  object 
 6   originalTitle   26359 non-null  object 
 7   isAdult         26359 non-null  int64  
 8   startYear       26359 non-null  object 
 9   endYear         26359 non-null  object 
 10  runtimeMinutes  26359 non-null  object 
 11  genres          26359 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 2.4+ MB


In [ ]:
#vérifier que la dataframe est bien filtré sur 'movie'

data_initial['titleType'].value_counts()

,count
titleType,
movie,26359


In [ ]:
#vérifier la prise en compte des filtres genres
data_initial['genres'].value_counts()

,count
genres,
Drama,24039
"Comedy,Drama",2320


In [ ]:
#vérifier si les films 'adult on bien été exclus
data_initial['isAdult'].value_counts()

,count
isAdult,
0,26359


In [ ]:
#Contrôle des années

data_initial['startYear'].value_counts().sort_index()

,count
startYear,
1960,620
1961,593
1962,626
1963,610
1964,611
1965,642
1966,634
1967,657
1968,795


In [ ]:
#Visualiser s'il y'a des films sans vote
data_initial['averageRating'].isna().value_counts()

# Il y a des films sans vote. Nous allons continuer à ajuster notre DataFrame en prenant en compte les autres KPI.
# À la fin, nous vérifierons de nouveau le pourcentage de films sans vote et consulterons le client pour décider de la marche à suivre si ce pourcentage dépasse 30 %.

,count
averageRating,
False,14407
True,11952


**Filtrer la dataframe initial pour tenir compte du KPI sur la durée :**

*   runtimeMinutes = entre 90 et 120

In [ ]:
#dans la colonne 'runtimeMinutes' on a une valeur '\N' non exploitable
# => il faut remplacer '\N' par 0 et convertir toutes les valeurs en int => utilisation de Pandas

data_initial['runtimeMinutes'].value_counts()

,count
runtimeMinutes,
\N,7529
90,1213
100,703
95,581
85,569
...,...
241,1
35,1
22,1


In [ ]:
#transformation de la colonne result_title_basics['runtimeMinutes'] : remplacer '\N' par 0 et convertir toutes les valeurs en int

data_initial['runtimeMinutes_int']= data_initial['runtimeMinutes'].replace('\\N',0).astype(int)
data_initial['runtimeMinutes_int']

,runtimeMinutes_int
0,100
1,88
2,80
3,106
4,128
...,...
26354,0
26355,0
26356,90
26357,85


In [ ]:
# contrôle du type
data_initial['runtimeMinutes_int'].dtypes

dtype('int64')

In [ ]:
# DataFrame comprenant : filtre 'movie' + filtre startYear + filtre genres + filtre rutimeMinutes

data_imdb=data_initial[((data_initial['runtimeMinutes_int'] >= 90) & (data_initial['runtimeMinutes_int'] <= 120))]
data_imdb

,tconst,averageRating,numVotes,tconst_1,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,runtimeMinutes_int
0,tt0052614,7.0,78.0,tt0052614,movie,Le bel âge,Le bel âge,0,1960,\N,100,"Comedy,Drama",100
3,tt0053570,7.6,1449.0,tt0053570,movie,Adua e le compagne,Adua e le compagne,0,1960,\N,106,"Comedy,Drama",106
5,tt0053643,7.3,1975.0,tt0053643,movie,Bell' Antonio,Il bell'Antonio,0,1960,\N,105,"Comedy,Drama",105
6,tt0053665,6.8,128.0,tt0053665,movie,Bonchi,Bonchi,0,1960,\N,105,"Comedy,Drama",105
7,tt0053671,6.6,187.0,tt0053671,movie,Boulevard,Boulevard,0,1960,\N,95,"Comedy,Drama",95
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26343,None,NaN,NaN,tt0312714,movie,Ganga Chiloner Pankhi,Ganga Chiloner Pankhi,0,1975,\N,105,Drama,105
26345,None,NaN,NaN,tt0312747,movie,Sorrow Under the Heavens,Haneulalae seulpeumi,0,1978,\N,100,Drama,100
26350,None,NaN,NaN,tt0312796,movie,How Can I Forget?,Ijeul suga isseulga,0,1970,\N,100,Drama,100
26352,None,NaN,NaN,tt0312825,movie,Izuko e,Izuko e,0,1966,\N,91,Drama,91


In [ ]:
data_imdb

,tconst,averageRating,numVotes,tconst_1,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,runtimeMinutes_int
0,tt0052614,7.0,78.0,tt0052614,movie,Le bel âge,Le bel âge,0,1960,\N,100,"Comedy,Drama",100
3,tt0053570,7.6,1449.0,tt0053570,movie,Adua e le compagne,Adua e le compagne,0,1960,\N,106,"Comedy,Drama",106
5,tt0053643,7.3,1975.0,tt0053643,movie,Bell' Antonio,Il bell'Antonio,0,1960,\N,105,"Comedy,Drama",105
6,tt0053665,6.8,128.0,tt0053665,movie,Bonchi,Bonchi,0,1960,\N,105,"Comedy,Drama",105
7,tt0053671,6.6,187.0,tt0053671,movie,Boulevard,Boulevard,0,1960,\N,95,"Comedy,Drama",95
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26343,None,NaN,NaN,tt0312714,movie,Ganga Chiloner Pankhi,Ganga Chiloner Pankhi,0,1975,\N,105,Drama,105
26345,None,NaN,NaN,tt0312747,movie,Sorrow Under the Heavens,Haneulalae seulpeumi,0,1978,\N,100,Drama,100
26350,None,NaN,NaN,tt0312796,movie,How Can I Forget?,Ijeul suga isseulga,0,1970,\N,100,Drama,100
26352,None,NaN,NaN,tt0312825,movie,Izuko e,Izuko e,0,1966,\N,91,Drama,91


In [ ]:
#vérifier les films sans vote
data_imdb['averageRating'].isna().value_counts()

,count
averageRating,
False,6957
True,2551


In [ ]:
data_imdb['averageRating'].isna().value_counts(normalize=True)*100

# le pourcentage de valeur nul est de 26,83% => vu avec le client pour ne pas remplacer les valeurs manquantes. Il faut les supprimer de la dataframe

,proportion
averageRating,
False,73.169962
True,26.830038


#**Dataframe IMDb finale respectant les KPI suivants :**

*   **titleType :  Movie**
*   **startYear :  Années 1960-70 et Nouveautés/ Avant-Premières (2022 et 2024)**
*   **genres :  Drama & Comedy, Drama**
*   **rutimeMinutes :  entre 90 et 120**
*   **rate :  averageRating > 7**








In [ ]:
# DataFrame finale : filtre Movie + filtre startYear + filtre genres + filtre rutimeMinutes + filtre rate

data_imdb=data_imdb[data_imdb['averageRating']>=7]

data_imdb

,tconst,averageRating,numVotes,tconst_1,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,runtimeMinutes_int
0,tt0052614,7.0,78.0,tt0052614,movie,Le bel âge,Le bel âge,0,1960,\N,100,"Comedy,Drama",100
3,tt0053570,7.6,1449.0,tt0053570,movie,Adua e le compagne,Adua e le compagne,0,1960,\N,106,"Comedy,Drama",106
5,tt0053643,7.3,1975.0,tt0053643,movie,Bell' Antonio,Il bell'Antonio,0,1960,\N,105,"Comedy,Drama",105
15,tt0054074,7.1,222.0,tt0054074,movie,My School Chum,Mein Schulfreund,0,1960,\N,94,"Comedy,Drama",94
17,tt0054242,7.3,959.0,tt0054242,movie,The Passionate Thief,Risate di gioia,0,1960,\N,106,"Comedy,Drama",106
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26155,tt0288653,7.2,53.0,tt0288653,movie,To nisi tis Afroditis,To nisi tis Afroditis,0,1965,\N,95,Drama,95
26161,tt0288797,7.3,15.0,tt0288797,movie,Sabaruddin Tukang Kasut,Sabaruddin Tukang Kasut,0,1966,\N,109,Drama,109
26232,tt0290265,8.5,12.0,tt0290265,movie,Ping Pong,Ping Pong,0,1968,\N,98,Drama,98
26288,tt0311583,7.2,263.0,tt0311583,movie,Night of the Hunchback,Shabe ghuzi,0,1965,\N,91,"Comedy,Drama",91


**Contrôle de la DATAFRAME**

In [ ]:
#controle de la DATA

data_imdb.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2241 entries, 0 to 26289
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tconst              2241 non-null   object 
 1   averageRating       2241 non-null   float64
 2   numVotes            2241 non-null   float64
 3   tconst_1            2241 non-null   object 
 4   titleType           2241 non-null   object 
 5   primaryTitle        2241 non-null   object 
 6   originalTitle       2241 non-null   object 
 7   isAdult             2241 non-null   int64  
 8   startYear           2241 non-null   object 
 9   endYear             2241 non-null   object 
 10  runtimeMinutes      2241 non-null   object 
 11  genres              2241 non-null   object 
 12  runtimeMinutes_int  2241 non-null   int64  
dtypes: float64(2), int64(2), object(9)
memory usage: 309.6+ KB


In [ ]:
data_imdb.columns

Index(['tconst', 'averageRating', 'numVotes', 'tconst_1', 'titleType',
       'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear',
       'runtimeMinutes', 'genres', 'runtimeMinutes_int'],
      dtype='object')

In [ ]:
data_imdb_final = data_imdb[['tconst','averageRating','titleType','originalTitle','startYear','runtimeMinutes','genres','runtimeMinutes_int']]
data_imdb_final

,tconst,averageRating,titleType,originalTitle,startYear,runtimeMinutes,genres,runtimeMinutes_int
0,tt0052614,7.0,movie,Le bel âge,1960,100,"Comedy,Drama",100
3,tt0053570,7.6,movie,Adua e le compagne,1960,106,"Comedy,Drama",106
5,tt0053643,7.3,movie,Il bell'Antonio,1960,105,"Comedy,Drama",105
15,tt0054074,7.1,movie,Mein Schulfreund,1960,94,"Comedy,Drama",94
17,tt0054242,7.3,movie,Risate di gioia,1960,106,"Comedy,Drama",106
...,...,...,...,...,...,...,...,...
26155,tt0288653,7.2,movie,To nisi tis Afroditis,1965,95,Drama,95
26161,tt0288797,7.3,movie,Sabaruddin Tukang Kasut,1966,109,Drama,109
26232,tt0290265,8.5,movie,Ping Pong,1968,98,Drama,98
26288,tt0311583,7.2,movie,Shabe ghuzi,1965,91,"Comedy,Drama",91


#**MERGE avec TMDB**

**1. Analyse de la base de donnée TMDB**

In [ ]:
import pandas as pd

# DataFrame TMDB
df_tmdb= pd.read_csv('tmdb_full.csv')
df_tmdb.head()

<ipython-input-20-0a5a65a68cd6>:4: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmdb= pd.read_csv('tmdb_full.csv')


,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,production_companies_name,production_companies_country
0,False,/dvQj1GBZAZirz1skEEZyWH2ZqQP.jpg,0,['Comedy'],NaN,3924,tt0029927,en,Blondie,Blondie and Dagwood are about to celebrate the...,...,70,['en'],Released,The favorite comic strip of millions at last o...,Blondie,False,7.214,7,['Columbia Pictures'],['US']
1,False,NaN,0,['Adventure'],NaN,6124,tt0011436,de,Der Mann ohne Namen,NaN,...,420,[],Released,NaN,"Peter Voss, Thief of Millions",False,0.000,0,[],[]
2,False,/uJlc4aNPF3Y8yAqahJTKBwgwPVW.jpg,0,"['Drama', 'Romance']",NaN,8773,tt0055747,fr,L'Amour à vingt ans,Love at Twenty unites five directors from five...,...,110,"['it', 'ja', 'pl', 'fr', 'de']",Released,The Intimate Secrets of Young Lovers,Love at Twenty,False,6.700,41,"['Ulysse Productions', 'Unitec Films', 'Cinese...","['', 'NZ', 'IT', 'JP', 'DE', 'PL', '']"
3,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,0,"['Drama', 'Comedy', 'Crime']",NaN,2,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,...,73,['fi'],Released,NaN,Ariel,False,7.046,248,['Villealfa Filmproductions'],['FI']
4,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,0,"['Drama', 'Comedy', 'Romance']",NaN,3,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",...,76,['en'],Released,NaN,Shadows in Paradise,False,7.182,269,['Villealfa Filmproductions'],['FI']


In [ ]:
df_tmdb.columns

Index(['adult', 'backdrop_path', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'production_companies_name',
       'production_companies_country'],
      dtype='object')

In [ ]:
df_tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309572 entries, 0 to 309571
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   adult                         309572 non-null  bool   
 1   backdrop_path                 151760 non-null  object 
 2   budget                        309572 non-null  int64  
 3   genres                        309572 non-null  object 
 4   homepage                      44262 non-null   object 
 5   id                            309572 non-null  int64  
 6   imdb_id                       309572 non-null  object 
 7   original_language             309572 non-null  object 
 8   original_title                309572 non-null  object 
 9   overview                      282512 non-null  object 
 10  popularity                    309572 non-null  float64
 11  poster_path                   264159 non-null  object 
 12  production_countries          309572 non-nul

In [ ]:
#vérifier s'il y a des film sans info dans 'orginal_language'
df_tmdb['original_language'].isna().sum()

0

In [ ]:
#selection des colonnes que nous souhaitons
data_tmdb = df_tmdb[['imdb_id','original_title','original_language','overview','poster_path']]
data_tmdb

,imdb_id,original_title,original_language,overview,poster_path
0,tt0029927,Blondie,en,Blondie and Dagwood are about to celebrate the...,/zBiHKhXklvTFwj4M1uEUcQGAVJ.jpg
1,tt0011436,Der Mann ohne Namen,de,NaN,/6xUbUCvndklbGVYiljHr34NTxSl.jpg
2,tt0055747,L'Amour à vingt ans,fr,Love at Twenty unites five directors from five...,/aup2QCYCsyEeQfpboXy0f4uj8aE.jpg
3,tt0094675,Ariel,fi,Taisto Kasurinen is a Finnish coal miner whose...,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg
4,tt0092149,Varjoja paratiisissa,fi,"An episode in the life of Nikander, a garbage ...",/nj01hspawPof0mJmlgfjuLyJuRN.jpg
...,...,...,...,...,...
309567,tt24962414,重裝戰警,zh,The film tells the story of the heavy-duty pol...,/tzF2BndyY3HIf2KjJR1FBYoIqTp.jpg
309568,tt17352384,Гранит,ru,Mozambique requests from Russia is being helpe...,/nqMk6TIUwqSMRc6QPKYguNr9Ql0.jpg
309569,tt8223104,"Marcelino, el mejor payaso del mundo",es,"An account of the life, work and many triumphs...",/btNlb2H3NXuBCN5J1FB9IVddYR.jpg
309570,tt14213656,Cheating Hitler: Surviving the Holocaust,en,"Three Canadian Holocaust survivors, with unans...",/uLfathKx9hauVKdasJc8A720kvb.jpg


**1. Filtrer la dataframe TMDB pour tenir compte du KPI sur la nationalité des films :**

*   original_language = fr

In [ ]:
# filtrer le FR

data_tmdb_fr= data_tmdb[data_tmdb['original_language'] == 'fr']
data_tmdb_fr

,imdb_id,original_title,original_language,overview,poster_path
2,tt0055747,L'Amour à vingt ans,fr,Love at Twenty unites five directors from five...,/aup2QCYCsyEeQfpboXy0f4uj8aE.jpg
76,tt0108394,Trois couleurs : Bleu,fr,Julie is haunted by her grief after living thr...,/33wsWxzsNstI8N7dvuwzFmj1qBd.jpg
77,tt0111507,Trois couleurs : Blanc,fr,Polish immigrant Karol Karol finds himself out...,/fdIet3NSa27gobMbaUml66oCQNT.jpg
78,tt0111495,Trois couleurs : Rouge,fr,"Valentine, a student model in Geneva, struggle...",/JHmsBiX1tjCKqAul1lzC20WcAW.jpg
112,tt0053198,Les Quatre Cents Coups,fr,"For young Parisian boy Antoine Doinel, life is...",/12PuU23kkDLvTd0nb8hMlE3oShB.jpg
...,...,...,...,...,...
309438,tt1804503,Conscience de magistrat,fr,The Magistrate's Conscience - A drama from Pat...,NaN
309440,tt1003393,Affaires de cœur,fr,A near-abstract animation telling a simple rom...,NaN
309441,tt0436841,Transformations élastiques,fr,A woman enters a room with a man. She creates...,NaN
309443,tt0277596,Le baromètre de la fidélité,fr,This Max Linder short opens with him and his g...,/jkeqURIMeo68IxtrreV2TKW0xeD.jpg


In [ ]:
data_tmdb_fr.columns

Index(['imdb_id', 'original_title', 'original_language', 'overview',
       'poster_path'],
      dtype='object')

In [ ]:
data_tmdb_fr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18383 entries, 2 to 309473
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   imdb_id            18383 non-null  object
 1   original_title     18383 non-null  object
 2   original_language  18383 non-null  object
 3   overview           15685 non-null  object
 4   poster_path        16381 non-null  object
dtypes: object(5)
memory usage: 861.7+ KB


**2. Fusion avec la dataframe 'data_imdb_final'**

In [ ]:
data_imdb_final.columns

Index(['tconst', 'averageRating', 'titleType', 'originalTitle', 'startYear',
       'runtimeMinutes', 'genres', 'runtimeMinutes_int'],
      dtype='object')

In [ ]:
data_tmdb_fr.columns

Index(['imdb_id', 'original_title', 'original_language', 'overview',
       'poster_path'],
      dtype='object')

In [ ]:
#fusion

data_imdb_tmdb = pd.merge(data_imdb_final,
                          data_tmdb_fr,
                          how='inner',
                          left_on= 'tconst',
                          right_on= 'imdb_id')
data_imdb_tmdb

,tconst,averageRating,titleType,originalTitle,startYear,runtimeMinutes,genres,runtimeMinutes_int,imdb_id,original_title,original_language,overview,poster_path
0,tt0052614,7.0,movie,Le bel âge,1960,100,"Comedy,Drama",100,tt0052614,Le Bel Âge,fr,"Steph, Jean-Claude and Jacques work in a Paris...",/98Z5yPoNIm8sQeyep5cpr5NHov9.jpg
1,tt0053279,7.2,movie,Le signe du lion,1962,103,Drama,103,tt0053279,Le Signe du Lion,fr,An American in Paris lives by sponging off his...,/b5jY3jyISYMY0ReGjqtbgaPq7dr.jpg
2,tt0055910,7.8,movie,Les dimanches de Ville d'Avray,1962,111,Drama,111,tt0055910,Cybèle ou les dimanches de ville d'Avray,fr,The tragic story of a young orphan girl who is...,/9L5DtYYlO3yrg2qtTCxROLnLl2V.jpg
3,tt0056636,7.4,movie,Un singe en hiver,1962,105,"Comedy,Drama",105,tt0056636,Un singe en hiver,fr,The story of a man who has stopped drinking- a...,/g0NxvdRRmuelSxc0NsmsgfZJrgi.jpg
4,tt0056581,7.1,movie,Thérèse Desqueyroux,1962,109,Drama,109,tt0056581,Thérèse Desqueyroux,fr,"Thérèse is living in a provincial town, unhapp...",/xqpxhzrJzKwJTcEcm7XIL70QrPD.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,tt27531109,7.0,movie,Le ravissement,2023,97,Drama,97,tt27531109,Le Ravissement,fr,"Lydia, a midwife very invested in her work, is...",/ld7Gdzj0prhCRau33zdyXetoKjD.jpg
75,tt9660502,7.8,movie,Close,2022,104,Drama,104,tt9660502,Close,fr,Two thirteen year-olds have always been incred...,/j7ReE0UZ9sgtfLvzszMgQw8Mxb2.jpg
76,tt0144785,7.6,movie,Les beaux dimanches,1974,93,Drama,93,tt0144785,Les beaux dimanches,fr,NaN,/nW5dQ8nhs3BVSqzWpigf6q4F6yg.jpg
77,tt0188266,7.1,movie,Utopia,1978,92,Drama,92,tt0188266,Utopia,fr,After getting painfully separated from the wom...,/6vPsJKO4OBhn3rQngt1LMD2NY4i.jpg


**3. Dataframe IMDb fusionnée avec la Dataframe TMDB**

In [ ]:
#film sans Posters => à supprimer car nous souhaitons avoir les images pour le site de recommandations
data_imdb_tmdb['poster_path'].isna().value_counts()

,count
poster_path,
False,76
True,3


In [ ]:
#supprimer les valeurs NA dans la colonne ['poster_path']
data_imdb_tmdb= data_imdb_tmdb.dropna(subset=['poster_path'])
data_imdb_tmdb

,tconst,averageRating,titleType,originalTitle,startYear,runtimeMinutes,genres,runtimeMinutes_int,imdb_id,original_title,original_language,overview,poster_path
0,tt0052614,7.0,movie,Le bel âge,1960,100,"Comedy,Drama",100,tt0052614,Le Bel Âge,fr,"Steph, Jean-Claude and Jacques work in a Paris...",/98Z5yPoNIm8sQeyep5cpr5NHov9.jpg
1,tt0053279,7.2,movie,Le signe du lion,1962,103,Drama,103,tt0053279,Le Signe du Lion,fr,An American in Paris lives by sponging off his...,/b5jY3jyISYMY0ReGjqtbgaPq7dr.jpg
2,tt0055910,7.8,movie,Les dimanches de Ville d'Avray,1962,111,Drama,111,tt0055910,Cybèle ou les dimanches de ville d'Avray,fr,The tragic story of a young orphan girl who is...,/9L5DtYYlO3yrg2qtTCxROLnLl2V.jpg
3,tt0056636,7.4,movie,Un singe en hiver,1962,105,"Comedy,Drama",105,tt0056636,Un singe en hiver,fr,The story of a man who has stopped drinking- a...,/g0NxvdRRmuelSxc0NsmsgfZJrgi.jpg
4,tt0056581,7.1,movie,Thérèse Desqueyroux,1962,109,Drama,109,tt0056581,Thérèse Desqueyroux,fr,"Thérèse is living in a provincial town, unhapp...",/xqpxhzrJzKwJTcEcm7XIL70QrPD.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,tt27531109,7.0,movie,Le ravissement,2023,97,Drama,97,tt27531109,Le Ravissement,fr,"Lydia, a midwife very invested in her work, is...",/ld7Gdzj0prhCRau33zdyXetoKjD.jpg
75,tt9660502,7.8,movie,Close,2022,104,Drama,104,tt9660502,Close,fr,Two thirteen year-olds have always been incred...,/j7ReE0UZ9sgtfLvzszMgQw8Mxb2.jpg
76,tt0144785,7.6,movie,Les beaux dimanches,1974,93,Drama,93,tt0144785,Les beaux dimanches,fr,NaN,/nW5dQ8nhs3BVSqzWpigf6q4F6yg.jpg
77,tt0188266,7.1,movie,Utopia,1978,92,Drama,92,tt0188266,Utopia,fr,After getting painfully separated from the wom...,/6vPsJKO4OBhn3rQngt1LMD2NY4i.jpg


In [ ]:
len(data_imdb_tmdb)

76

#**MERGE** **data_imdb_tmdb avec link_6 'title principals' puis link_1 'name.basic'**

**Objectif: récupérer le nom des acteurs, actrices et realisateurs**

**1. Consulter la dataset [title.principals](https://datasets.imdbws.com/title.principals.tsv.gz)**
 **et fusionner avec la dataframe data_imdb_tmdb**


In [ ]:
import pandas as pd
import duckdb

# Connexion DuckDB
con = duckdb.connect()

# Requête optimisée pour filtrer les catégories et limiter les colonnes extraites
query = f"""
    SELECT tconst, nconst, category
    FROM read_csv_auto('https://datasets.imdbws.com/title.principals.tsv.gz', delim='\t', quote='', ignore_errors=true)
    WHERE category IN ('actor', 'actress', 'director')
"""

# Exécuter et récupérer les données filtrées
link_title_principals = con.execute(query).fetchdf()

# Afficher les premières lignes pour validation
print(link_title_principals.head())

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

      tconst     nconst category
0  tt0000005  nm0443482    actor
1  tt0000005  nm0653042    actor
2  tt0000007  nm0179163    actor
3  tt0000007  nm0183947    actor
4  tt0000008  nm0653028    actor


In [ ]:
# fusion avec la dataframe

data_imdb_tmdb_final = pd.merge(data_imdb_tmdb,
                link_title_principals,
                how = 'left',
                on = 'tconst')

data_imdb_tmdb_final

,tconst,averageRating,titleType,originalTitle,startYear,runtimeMinutes,genres,runtimeMinutes_int,imdb_id,original_title,original_language,overview,poster_path,nconst,category
0,tt0052614,7.0,movie,Le bel âge,1960,100,"Comedy,Drama",100,tt0052614,Le Bel Âge,fr,"Steph, Jean-Claude and Jacques work in a Paris...",/98Z5yPoNIm8sQeyep5cpr5NHov9.jpg,nm0656496,actor
1,tt0052614,7.0,movie,Le bel âge,1960,100,"Comedy,Drama",100,tt0052614,Le Bel Âge,fr,"Steph, Jean-Claude and Jacques work in a Paris...",/98Z5yPoNIm8sQeyep5cpr5NHov9.jpg,nm0895644,actor
2,tt0052614,7.0,movie,Le bel âge,1960,100,"Comedy,Drama",100,tt0052614,Le Bel Âge,fr,"Steph, Jean-Claude and Jacques work in a Paris...",/98Z5yPoNIm8sQeyep5cpr5NHov9.jpg,nm0232540,actor
3,tt0052614,7.0,movie,Le bel âge,1960,100,"Comedy,Drama",100,tt0052614,Le Bel Âge,fr,"Steph, Jean-Claude and Jacques work in a Paris...",/98Z5yPoNIm8sQeyep5cpr5NHov9.jpg,nm0108400,actor
4,tt0052614,7.0,movie,Le bel âge,1960,100,"Comedy,Drama",100,tt0052614,Le Bel Âge,fr,"Steph, Jean-Claude and Jacques work in a Paris...",/98Z5yPoNIm8sQeyep5cpr5NHov9.jpg,nm0261161,actor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,tt0202062,7.3,movie,Le vieux pays où Rimbaud est mort,1977,114,Drama,114,tt0202062,Le vieux pays où Rimbaud est mort,fr,A middle-aged man travels to France and is dis...,/nGInAItTJk5gNenj2JxPoW41YDh.jpg,nm0674877,actor
792,tt0202062,7.3,movie,Le vieux pays où Rimbaud est mort,1977,114,Drama,114,tt0202062,Le vieux pays où Rimbaud est mort,fr,A middle-aged man travels to France and is dis...,/nGInAItTJk5gNenj2JxPoW41YDh.jpg,nm0063194,actor
793,tt0202062,7.3,movie,Le vieux pays où Rimbaud est mort,1977,114,Drama,114,tt0202062,Le vieux pays où Rimbaud est mort,fr,A middle-aged man travels to France and is dis...,/nGInAItTJk5gNenj2JxPoW41YDh.jpg,nm0077749,actor
794,tt0202062,7.3,movie,Le vieux pays où Rimbaud est mort,1977,114,Drama,114,tt0202062,Le vieux pays où Rimbaud est mort,fr,A middle-aged man travels to France and is dis...,/nGInAItTJk5gNenj2JxPoW41YDh.jpg,nm0080532,actor


**2. Consulter la dataset [name.basics](https://datasets.imdbws.com/name.basics.tsv.gz)**
 **et fusionner avec la dataframe data**

In [ ]:
import duckdb

# Connexion à DuckDB
con = duckdb.connect()

# Requête SQL pour lire et filtrer les colonnes nécessaires
query = """
    SELECT nconst, primaryName
    FROM read_csv_auto('https://datasets.imdbws.com/name.basics.tsv.gz', delim='\t', quote='', ignore_errors=true)
    WHERE (primaryProfession LIKE '%actor%') OR (primaryProfession LIKE '%actress%') OR (primaryProfession LIKE '%director%')
"""

# Exécution de la requête et chargement dans un DataFrame Pandas
link_acteur_name = con.execute(query).fetchdf()

# Affichage des premières lignes pour validation
print(link_acteur_name.head())

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

      nconst      primaryName
0  nm0000001     Fred Astaire
1  nm0000004     John Belushi
2  nm0000005   Ingmar Bergman
3  nm0000007  Humphrey Bogart
4  nm0000008    Marlon Brando


In [ ]:
# fusion avec la dataframe

data_imdb_tmdb_final = pd.merge(data_imdb_tmdb_final,
                                 link_acteur_name,
                                 how = 'left',
                                 on = 'nconst')

data_imdb_tmdb_final

,tconst,averageRating,titleType,originalTitle,startYear,runtimeMinutes,genres,runtimeMinutes_int,imdb_id,original_title,original_language,overview,poster_path,nconst,category,primaryName
0,tt0052614,7.0,movie,Le bel âge,1960,100,"Comedy,Drama",100,tt0052614,Le Bel Âge,fr,"Steph, Jean-Claude and Jacques work in a Paris...",/98Z5yPoNIm8sQeyep5cpr5NHov9.jpg,nm0656496,actor,Marcello Pagliero
1,tt0052614,7.0,movie,Le bel âge,1960,100,"Comedy,Drama",100,tt0052614,Le Bel Âge,fr,"Steph, Jean-Claude and Jacques work in a Paris...",/98Z5yPoNIm8sQeyep5cpr5NHov9.jpg,nm0895644,actor,Boris Vian
2,tt0052614,7.0,movie,Le bel âge,1960,100,"Comedy,Drama",100,tt0052614,Le Bel Âge,fr,"Steph, Jean-Claude and Jacques work in a Paris...",/98Z5yPoNIm8sQeyep5cpr5NHov9.jpg,nm0232540,actor,Jacques Doniol-Valcroze
3,tt0052614,7.0,movie,Le bel âge,1960,100,"Comedy,Drama",100,tt0052614,Le Bel Âge,fr,"Steph, Jean-Claude and Jacques work in a Paris...",/98Z5yPoNIm8sQeyep5cpr5NHov9.jpg,nm0108400,actor,Jean-Claude Brialy
4,tt0052614,7.0,movie,Le bel âge,1960,100,"Comedy,Drama",100,tt0052614,Le Bel Âge,fr,"Steph, Jean-Claude and Jacques work in a Paris...",/98Z5yPoNIm8sQeyep5cpr5NHov9.jpg,nm0261161,actor,Giani Esposito
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,tt0202062,7.3,movie,Le vieux pays où Rimbaud est mort,1977,114,Drama,114,tt0202062,Le vieux pays où Rimbaud est mort,fr,A middle-aged man travels to France and is dis...,/nGInAItTJk5gNenj2JxPoW41YDh.jpg,nm0674877,actor,François Perrot
792,tt0202062,7.3,movie,Le vieux pays où Rimbaud est mort,1977,114,Drama,114,tt0202062,Le vieux pays où Rimbaud est mort,fr,A middle-aged man travels to France and is dis...,/nGInAItTJk5gNenj2JxPoW41YDh.jpg,nm0063194,actor,Jean-François Bayonne
793,tt0202062,7.3,movie,Le vieux pays où Rimbaud est mort,1977,114,Drama,114,tt0202062,Le vieux pays où Rimbaud est mort,fr,A middle-aged man travels to France and is dis...,/nGInAItTJk5gNenj2JxPoW41YDh.jpg,nm0077749,actor,Fernand Berset
794,tt0202062,7.3,movie,Le vieux pays où Rimbaud est mort,1977,114,Drama,114,tt0202062,Le vieux pays où Rimbaud est mort,fr,A middle-aged man travels to France and is dis...,/nGInAItTJk5gNenj2JxPoW41YDh.jpg,nm0080532,actor,Tomasz Bialkowski


In [ ]:
data_imdb_tmdb_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 796 entries, 0 to 795
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tconst              796 non-null    object 
 1   averageRating       796 non-null    float64
 2   titleType           796 non-null    object 
 3   originalTitle       796 non-null    object 
 4   startYear           796 non-null    object 
 5   runtimeMinutes      796 non-null    object 
 6   genres              796 non-null    object 
 7   runtimeMinutes_int  796 non-null    int64  
 8   imdb_id             796 non-null    object 
 9   original_title      796 non-null    object 
 10  original_language   796 non-null    object 
 11  overview            759 non-null    object 
 12  poster_path         796 non-null    object 
 13  nconst              796 non-null    object 
 14  category            796 non-null    object 
 15  primaryName         795 non-null    object 
dtypes: float

In [ ]:
data_imdb_tmdb_final.isna().sum()

,0
tconst,0
averageRating,0
titleType,0
originalTitle,0
startYear,0
runtimeMinutes,0
genres,0
runtimeMinutes_int,0
imdb_id,0
original_title,0


In [ ]:
data_imdb_tmdb_final[data_imdb_tmdb_final['primaryName'].isna()]
# ok n'apparaît pas car PrincipalProfession = composer,writer,music_department. On n'a pas acteur, actrice ou réalisateur

,tconst,averageRating,titleType,originalTitle,startYear,runtimeMinutes,genres,runtimeMinutes_int,imdb_id,original_title,original_language,overview,poster_path,nconst,category,primaryName
647,tt0284376,7.0,movie,Où êtes-vous donc?,1969,96,Drama,96,tt0284376,Où êtes-vous donc?,fr,This experimental feature-length drama by Gill...,/5oRNuBjDU1SrkNUUIGWehDj0n08.jpg,nm0893170,actor,NaN


#**Dataframe Finale**

In [ ]:
#Sélection des colonnes pertinentes

data_final = data_imdb_tmdb_final[['tconst','titleType','original_title','startYear','runtimeMinutes','genres','original_language','overview','averageRating','category','primaryName','poster_path']]
data_final

,tconst,titleType,original_title,startYear,runtimeMinutes,genres,original_language,overview,averageRating,category,primaryName,poster_path
0,tt0052614,movie,Le Bel Âge,1960,100,"Comedy,Drama",fr,"Steph, Jean-Claude and Jacques work in a Paris...",7.0,actor,Marcello Pagliero,/98Z5yPoNIm8sQeyep5cpr5NHov9.jpg
1,tt0052614,movie,Le Bel Âge,1960,100,"Comedy,Drama",fr,"Steph, Jean-Claude and Jacques work in a Paris...",7.0,actor,Boris Vian,/98Z5yPoNIm8sQeyep5cpr5NHov9.jpg
2,tt0052614,movie,Le Bel Âge,1960,100,"Comedy,Drama",fr,"Steph, Jean-Claude and Jacques work in a Paris...",7.0,actor,Jacques Doniol-Valcroze,/98Z5yPoNIm8sQeyep5cpr5NHov9.jpg
3,tt0052614,movie,Le Bel Âge,1960,100,"Comedy,Drama",fr,"Steph, Jean-Claude and Jacques work in a Paris...",7.0,actor,Jean-Claude Brialy,/98Z5yPoNIm8sQeyep5cpr5NHov9.jpg
4,tt0052614,movie,Le Bel Âge,1960,100,"Comedy,Drama",fr,"Steph, Jean-Claude and Jacques work in a Paris...",7.0,actor,Giani Esposito,/98Z5yPoNIm8sQeyep5cpr5NHov9.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
791,tt0202062,movie,Le vieux pays où Rimbaud est mort,1977,114,Drama,fr,A middle-aged man travels to France and is dis...,7.3,actor,François Perrot,/nGInAItTJk5gNenj2JxPoW41YDh.jpg
792,tt0202062,movie,Le vieux pays où Rimbaud est mort,1977,114,Drama,fr,A middle-aged man travels to France and is dis...,7.3,actor,Jean-François Bayonne,/nGInAItTJk5gNenj2JxPoW41YDh.jpg
793,tt0202062,movie,Le vieux pays où Rimbaud est mort,1977,114,Drama,fr,A middle-aged man travels to France and is dis...,7.3,actor,Fernand Berset,/nGInAItTJk5gNenj2JxPoW41YDh.jpg
794,tt0202062,movie,Le vieux pays où Rimbaud est mort,1977,114,Drama,fr,A middle-aged man travels to France and is dis...,7.3,actor,Tomasz Bialkowski,/nGInAItTJk5gNenj2JxPoW41YDh.jpg


In [ ]:
data_final['category'].value_counts()

,count
category,
actor,423
actress,296
director,77


In [ ]:
data_final['genres'].value_counts()

,count
genres,
Drama,606
"Comedy,Drama",190


In [ ]:
data_final['averageRating'].value_counts()

,count
averageRating,
7.1,204
7.3,129
7.0,113
7.2,98
7.4,89
7.5,58
7.8,45
7.6,23
7.7,11


In [ ]:
len(data_final['tconst'].unique())

76

In [ ]:
export_csv=data_final.to_csv('datafinalp2.csv', index=False)


In [ ]:
export_csv